# 一、建模（超参数选择）

## 数据集划分
- 将数据划分为训练集、验证集（validation set/development set/dev set）和测试集——训练集用于建模，验证集用于选取超参数，测试集用于测试模型效果。  
- 以往经验划分（小规模）数据集：
    - 70%训练集、30%测试集
    - 60%训练集、20%验证集、20%测试集  
- 但现代大数据量的数据集划分：
    - 验证集和训练集只需要留少量数据验证模型参数和测试精度，大部分数据保留在训练集用于建模
    - 比如1,000,000个数据，只需要10,000作为验证集，10,000作为测试集即可    
    
## 数据集来源
- 现代数据的训练集和测试集常常是不同来源不同分布，比如训练集下载于网络，测试集来自于真实用户
- 这种情况要保证验证集和测试集来自同一个来源，具有同样的分布
- 或者只用训练集和验证集，不要测试集  

## 机器学习基本策略
- 高偏置：关注训练集效果，可以尝试：
    - 更大的网络
    - 更长的训练时间  
    
- 高次项：关注验证集效果，可以尝试：
    - 更多数据
    - 正则化  
    
# 正则化

## logistic回归正则项
常用L2范数  

## 神经网络正则项
- [Frobenius norm](https://en.wikipedia.org/wiki/Matrix_norm#Frobenius_norm),有时也叫做weight decay
- 如何正则：比如用`tanh`作为激活函数的神经网络，如果正则项中$\lambda$很大，相应的初始化在0附近的权重W会变小，进而$z^{[l]}=W^{[l]}a^{[l]}+b^{[l]}$进入`tanh`的线性部分（0附近），从而把网络变成线性回归，网络过拟合的可能性变小  
- 何处加正则项：
    - 损失函数
    - 反向传播的权重矩阵更新项（梯度）

## Dropout
- 用于训练集，测试集不用
- 常用语CV领域，因为常常没有足够多的数据，会过拟合，而每个数据（图片），又有过多的特征（像素）  

## Data Augmentation
图像作为数据时，可以用水平变换，旋转等手段生成‘新的’图片作为新数据，更多数据以防止过拟合的方法，类似于一种正则化
## Early Stopping

# 优化

## Normalization
以2维为例，未做normalization的数据的损失函数可能是椭圆，normalization后变为圆，做梯度下降时收敛更快  

## 权重初始化
- `Wl = np.random.randn(shape) * np.sqrt(2/n_prev)`适用于激活函数为RELU
- Xavier initialization适用于激活函数为`tanh`
- Yoshua Bengio提出的初始化$\sqrt{\frac{2}{n^{[l]}+n^{[l-1]}}}$

# 二、Optimization algorithms

## Mini batch SGD
- 在mini batch SGD中，对所有mini batch，即整个训练集进行一次遍历，叫做一个epoch，其中的步数（iteration），即为mini batch的个数
- mini batch大小（两个极端）对收敛速度的影响：
    - m=m(整个样本大小），mini-batch变为BGD，基本朝全局最小值收敛，但每步循环变慢
    - m=1，mini-batch变为SGD，每步迭代速度快，但是噪声多，并失去向量化的意义  
- 尺寸选择：
    - 数据集小，直接用BGD
    - 数据集大，可以在64,128,256,512中选择试验
    - 适合CPU/GPU的内存大小  
- mini batch SGD一般比SGD和BGD都要快
- 生成mini batch需要经过shuffle和partition两步
    
## Momentum
- momentum可以看做是小球沿着梯度方向下山时产生的动量  

`for l in range L:`  
$$ \begin{cases}
v_{dW^{[l]}} = \beta v_{dW^{[l]}} + (1 - \beta) dW^{[l]} \\
W^{[l]} = W^{[l]} - \alpha v_{dW^{[l]}}
\end{cases}\tag{1}$$

$$\begin{cases}
v_{db^{[l]}} = \beta v_{db^{[l]}} + (1 - \beta) db^{[l]} \\
b^{[l]} = b^{[l]} - \alpha v_{db^{[l]}} 
\end{cases}\tag{2}$$  

- 一般$\beta$取值范围从0.8到0.999，一般模型取0.9是合适的
- 根据损失J调整$\beta$
- momentum适合SGD，miniBGD，BGD

## RMSprop（Root Mean Square prop）
$$\begin{cases}
S_dW = \beta S_dW + (1-\beta)dW^2 \\
W:=W-\alpha\frac{dW}{\sqrt{S_dW}}
\end{cases}\tag{3}$$
$$\begin{cases}
S_db = \beta S_db + (1-\beta)db^2 \\
b:=b-\alpha\frac{db}{\sqrt{S_db}}
\end{cases}\tag{4}$$  

减少梯度下降时因各个维度尺度不同而产生的震荡，作用类似于Momentum  

## Adam（Adaptive Moment estimation）
初始化$V_dW=0,S_dW=0.V_db=0,S_db=0$  

第t次循环：
$$V_dW=\beta_1 V_dW+(1-\beta_1)dW,V_db=\beta_1 V_db+(1-\beta_1)db\tag{Momentum$\beta_1$}$$
$$S_dW=\beta_2 S_dW+(1-\beta_2)dW^2,S_db=\beta_1 S_db+(1-\beta_2)db^2\tag{RMSprop$\beta_2$}$$
$$V^{corrected}_{dW}=\frac{V_{dW}}{1-\beta^t_1},V^{corrected}_{db}=\frac{V_{db}}{1-\beta^t_1}$$
$$S^{corrected}_{dW}=\frac{S_{dW}}{1-\beta^t_2},S^{corrected}_{db}=\frac{S_{db}}{1-\beta^t_2}$$
$$W:=W-\alpha\frac{V^{corrected}_{dW}}{\sqrt{S^{corrected}_{dW}}+\epsilon}$$
$$b:=b-\alpha\frac{V^{corrected}_{db}}{\sqrt{S^{corrected}_{db}}+\epsilon}$$  

- 超参数：
    - $\alpha$：需调整
    - $\beta_1$：一般取0.9
    - $\beta_2$：一般取0.99
    - $\epsilon$：一般取10e-8，为防止式子除0  
    
## learning rate decay
- 常用decay方法：
$$\alpha=\frac{1}{1+decay-rate*epoch-num}\alpha_0$$  

举例说明：比如初始学习率$\alpha_0=0.2$,`decay-rate=1`，则每个epoch学习率计算如下——  

|Epoch    |$\alpha$    |
|----------|-------------|
|1      |0.1       |
|2      |0.67       |
|3      |0.5        |
|4      |0.4        |  

- 另外几种decay：
    - $\alpha=0.95^{epoch-num}\cdot\alpha_0$指数decay
    - $\alpha=\frac{k}{\sqrt{epoch-num}}\cdot\alpha_0$
    - $\alpha=\frac{k}{\sqrt{t}}\cdot\alpha_0$
    - 经过数个iteration，把学习率减小一半  
    
## 局部最小值


# 三、超参数调整
- 按重要度分
    - 最重要的：$\alpha$
    - 次重要的：$\beta$、隐藏单元数、mini-batch大小
    - 第三重要的：隐藏层数、学习率decay
    - 用Adam的时候，$\beta_1$一般取0.9、$\beta2$一般取0.999，$\epsilon$一般取10e-8，不做过多调整  
    
- 选择超参数（多个），不要用grid search的方法，因为不知道哪个超参数对模型影响大。推荐随机选择调整
- 比如学习率，从0.0001到1之间选择值，不要在该区域以均匀分布随机选择值，要以对数几率随机选择值，即$\sigma = log(\alpha)，\sigma\in[-4,0]$，在[-4,0]的区域上随机选择$\sigma$
- 同样，选择$\beta$时，一般在0.9到0.999之间选择时，可以认为是选择$1-\beta$在0.1到0.001之间选择，同学习率选择原则  

## Normalizing activation(Batch-normalization)
### 算法
- 神经网络中在输入到激活函数之前的z值进行normalization
$$\mu=\frac{1}{m}\sum_i{z^{(i)}} \\
\sigma^2=\frac{1}{m}\sum_i(z^{(i)}-\mu)^2 \\
z_{normalized}^{(i)}=\frac{z^{(i)}-\mu}{\sqrt{\sigma^2+\epsilon}}$$  

- 但实际上$z^{(i)}$可能不属于同一个分布，所以提出用$\tilde{z}^{(i)}$代替：
$$\tilde{z}^{(i)}=\gamma z_{normalized}^{(i)} +\beta \\
(If:\ \gamma=\sqrt{\sigma^2+\epsilon},\beta=\mu; \\
then:\ \tilde{z}^{(i)}=z_{normalized}^{(i)})$$
在实践中，$\gamma$和$\beta$都是可学习需更新的参数（learnable parameters）,每层网络对应一组$\gamma$、$\beta$  

### 原理
- **由于在batch normalization中，$z^{(i)}$需要减去均值$\mu$，所以初始化中的$b^{(i)}$对最终进入激活函数的$\tilde{z}^{(i)}$值没有影响。一般初始化为0**
- 在backpropagation中，$\gamma$、$\beta$与权重同样计算梯度，乘以学习率后更新
- batch normalization相当于对每层输入激活函数的值进行一次normalize，与数据集X的normalization同理
- **特点**：
    - 每个mini-batch都减去了均值并除以方差，由于mini-batch的大小以及分布可能与总体样本会有差距，所以与Dropout类似，会给隐藏层的激活函数添加一些噪声
    - 也因为噪声，batch normalization会有少许regularization的效果
    - batch小（意味着batch多），噪声多，regularization效果大；batch大，噪声会小，regularization效果少  
    
### 测试集上的做法
估算每个mini-batch在某一层上的$\mu$、$\sigma$，并求均值，为输入这一层的测试集数据进行scaling  

## Softmax层
- 用于多分类的最终激活函数层，生成各个分类的概率——相对于hardmax：将输出分类的最大值设为1，其余为0的向量
- 两个分类的softmax退化为logistic回归